In [ ]:
# First of all, on Colab, switch runtime type to "GPU"
# -- under the Runtime tag, click 'change runtime type', select 'GPU' as Hardware accelerator. 

## 1. Mount Google Drive

Mount your Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Install and load necessary modules and pre-trained models

In this section, we will install the necessary packages, including spacy version 2.3.1 and scispacy, and the pre-trained NER models from scispacy. The scispacy package should be installed before installing spacy 2.3.1. Then, we will load the necessary modules and the pre-trained models. 

Note: when install spacy 2.3.1, an error may encounter: "ERROR: scispacy 0.4.0 has requirement spacy<3.1.0,>=3.0.0, but you'll have spacy 2.3.1 which is incompatible." and "ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible." These will not be problems to perform the following works as long as the package is successfully installed. 

In [ ]:
# Installations needed
!pip install scispacy
!pip install -U spacy==2.3.1
!pip install transformers
!pip install knockknock==0.1.7
!pip install sentencepiece
!pip install --index-url https://test.pypi.org/simple/ --no-deps biomarker-nlp

     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 13.5MB 201kB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 12.8MB 243kB/s 
     |████████████████████████████████| 194kB 58.6MB/s 
     |████████████████████████████████| 460kB 35.9MB/s 
     |████████████████████████████████| 9.1MB 48.0MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 624kB 47.3MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |███████████████████

     |████████████████████████████████| 1.2MB 27.4MB/s 
Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 51kB 7.1MB/s 


In [ ]:
# install pre-trained models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_craft_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_jnlpba_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bc5cdr_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bionlp13cg_md-0.3.0.tar.gz

     |████████████████████████████████| 17.0MB 1.2MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.2.4-cp37-none-any.whl size=17161109 sha256=b0cd91faf89de192db6e6c0573d4ba1e586280b98f79d57fa9717c576f293f4d
  Stored in directory: /root/.cache/pip/wheels/34/60/b9/fabd9c3eeba17ed66df745479f2fc502a6702755cb4a9632f2
Successfully built en-core-sci-sm
     |████████████████████████████████| 79.8MB 36kB/s 
  Created wheel for en-ner-craft-md: filename=en_ner_craft_md-0.3.0-cp37-none-any.whl size=79896619 sha256=fc1c06c240591211ea6e48437ef514bac881a7de960a1087721a4643cdc91b19
  Stored in directory: /root/.cache/pip/wheels/86/13/90/f455ef155601f78b8466208c276a8944f3bad5b164b4520e20
Successfully built en-ner-craft-md
     |████████████████████████████████| 79.8MB 38kB/s 
  Created wheel for en-ner-jnlpba-md: filename=en_ner_jnlpba_md-0.3.0-cp37-none-any.whl size=79887396 sha256=8412eba5fe5a1bdd2aba2eb76c5927601ab3aab05cc935b26202a3fc4d0b5af2
  Stored in directory: /root/.cache

In [ ]:
# import necessary packages
from lxml import etree
import requests
import lxml.html
from urllib.parse import urlparse
import nltk
from nltk import tokenize
import re
nltk.download('punkt')
import scispacy
import spacy
from spacy import displacy
from biomarker_nlp import biomarker_extraction
from biomarker_nlp import negation_cue_scope
# Load modules from Aditya and Suraj's NegBERT programs.
from biomarker_nlp.negation_negbert import *
# import pre-trained NER models
import en_core_sci_sm
import en_ner_craft_md
import en_ner_jnlpba_md
import en_ner_bc5cdr_md
import en_ner_bionlp13cg_md

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
# load the pre-trained NER models
nlp = en_core_sci_sm.load()
nlp_craft = en_ner_craft_md.load()
nlp_jnlpha = en_ner_jnlpba_md.load()
nlp_bc5cdr = en_ner_bc5cdr_md.load()
nlp_bionlp13cg = en_ner_bionlp13cg_md.load()

/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In the next chunk, we will load the two pre-trained negation models (negCue and negScope) by providing their google drive path. Before loading, you will first need to upload the two models to your google drive. 

For more details about downloading these two models, please see the project's [Homepage](https://github.com/Damonlin11/FDA-Target-Therapy-Label-Extraction) page. 

In [ ]:
# load Negbert pre-train models
modelCue = torch.load('/content/drive/MyDrive/RA_NLP_drug_label/Negation_extraction/modelCue') # change path to the model file on your Google drive
modelScope = torch.load('/content/drive/MyDrive/RA_NLP_drug_label/Negation_extraction/modelScope') # change path to the model file on your Google drive

## 3. Biomarkers extraction Program

The program will go through every unique FDA-approved targeted therapy listed on NCI and extract its biomarkers label on its NCI and DailyMed "INDICATIONS AND USAGE" section. The biomarkers label includes gene/protein, therapy, disease, drug label, NDC code, drugs that can be combined to use, drug/gene/protein that are negated, and indicators for, such as, first-line treatment. The biomarker labels are extracted in lists. Then, these lists are put in a data frame, and duplicate rows are removed to get the final dataset. There will be some duplicate rows because of the design of the program. So, we will drop those rows. 

How to explain the data in the final dataset?
As the text processing in the program is sentence-based, each row of the dataset, in general, represents the biomarkers information in a sentence. 

In [ ]:
# create empty lists to contain values
therapy_list = []
web_gov = 'https://www.cancer.gov'
geneProtein_label = []
therapy_label = []
drug_label = []
NDCCode_label = []
disease_label = []
combinationDrug_label = []
negGeProDr_label = []
firstLine = []
acceleratedApproval = []
acceleratedApprovalRate = []
metastatic = []

count = 0

# get the links to FDA-approved targeted therapies
url_nci = "https://www.cancer.gov/about-cancer/treatment/types/targeted-therapies/targeted-therapies-fact-sheet"
therapies = biomarker_extraction.targeted_therapy_url(url = url_nci)

for i in therapies:
    link = ""
    
    # make sure the URL is complete
    if web_gov in i:
        link = i
    else:
        link = web_gov + i
    
    # test if the URL is valid
    result = urlparse(link)
    if all([result.scheme, result.netloc, result.path]):
        therName = ""
        therBrName = ""
        
        ## NCI page

        # extract the therapy's name
        therName = biomarker_extraction.targeted_therapy_name(url= link)
        #print(therName)

        if len(therName) > 0:
            therName = therName[0]
            print(therName)
            
            # count the number of therapies that have been screened
            count += 1
            print(count)
            
            # check duplicate therapy
            if therName.lower() not in therapy_list:
                therapy_list.append(therName.lower())
                
                # extract the corresponding diseases (only for bold text, but there are a few non-bold diseases left)
                diseaseList = biomarker_extraction.therapy_disease(url = link)
                print(diseaseList)
                ## DailyMed page
                
                # obtain DailyMed link
                dailyMed = biomarker_extraction.drug_search_url(url = link)
                
                if len(dailyMed)>0:
                
                    dailyMedURL = dailyMed[0]
            
                    # make sure the dailymed url is valid
                    dailyMedResult = urlparse(dailyMedURL)
                    if all([dailyMedResult.scheme, dailyMedResult.netloc, dailyMedResult.path]):
                    
                        # check if the link is directed to drug info page or multiple drug search page
                        dailyMedRp = requests.get(dailyMedURL)
                        dailyMedPage = lxml.html.fromstring(dailyMedRp.text)

                        # multiple drug selection page
                        if 'dailymed/search' in dailyMedURL:
                            print('multiple choice')
                            #print(dailyMedURL)
                            
                            # multiple drug search page, get each drug link
                            searchResults = dailyMedPage.xpath("//div[@class = 'results-info']")
                            for s in range(1,len(searchResults)+1):
                              dailyMedDrugURL = dailyMedPage.xpath("(//div[@class = 'results-info'])[" + str(s) + "]//a[@class='drug-info-link']/@href")
                              #print(len(dailyMedDrugURL))
                              
                              # extract the drug infor url from multiple drug search page
                              if len(dailyMedDrugURL) > 0:
                                  dailyMedDrugURL = dailyMedDrugURL[0]
  
                                  # check it is a link to drug information page
                                  if 'dailymed/drugInfo' in dailyMedDrugURL:
                                    dailyMedURL = 'https://dailymed.nlm.nih.gov' + dailyMedDrugURL
                                    print(dailyMedURL)

                                    # extract the brand name
                                    therBrName = biomarker_extraction.drug_brand_label(dailyMedURL = dailyMedURL)
                                    brName = therBrName.split("-")[0]

                                    # extract the NDC code
                                    NDCCodes = biomarker_extraction.ndc_code(dailyMedURL = dailyMedURL)

                                    # diseases are detected
                                    if len(diseaseList) > 0:

                                      for d in diseaseList:
                                        print(d)

                                        # locate 'INDICATIONS AND USAGE' section on DailyMed and extract the text content of a certain cancer type. 
                                        # Use '\t' or ' ' + d to handle issue of disease with Non in front, e.g.'Non-Small Cell Lung Cancer' and 'Small Cell Lung Cancer'
                                        # 'Non-' in the disease list
                                        if any('non-' + d.lower() in dl.lower() for dl in diseaseList):
                                          dailyMedDisContentStr_1 = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = '\t' + d, header= False)
                                          dailyMedDisContentStr_2 = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = ' ' + d, header= False)
                                          if dailyMedDisContentStr_1:
                                            dailyMedDisContentStr = dailyMedDisContentStr_1
                                          elif dailyMedDisContentStr_2:
                                            dailyMedDisContentStr = dailyMedDisContentStr_2
                                          else:
                                            dailyMedDisContentStr = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = d, header= False)

                                        # 'Non-' not in the disease list 
                                        else: dailyMedDisContentStr = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = d, header= False)

                                        # get the whole section text of 'INDICATIONS AND USAGE'
                                        sectionContent = biomarker_extraction.section_content(dailyMedURL, section = 'INDICATIONS AND USAGE')

                                        contentStr = None
                                        if dailyMedDisContentStr:
                                          #print(dailyMedDisContentStr)
                                          contentStr = dailyMedDisContentStr

                                        else:
                                          if sectionContent:
                                            if d.lower() in sectionContent.lower():
                                              contentStr = sectionContent

                                        # text information related to the disease
                                        if contentStr:
                                          print('has disease content')
                                          # check if it is accelerated approval and by response rate
                                          accApprRate = ''
                                          if biomarker_extraction.is_accelerated_approval_rate(contentStr):
                                            accApprRate = 'Yes'

                                          # check if it is accelerated approval
                                          accAppr = ''
                                          if biomarker_extraction.is_accelerated_approval(contentStr):
                                            accAppr = 'Yes'

                                          # split into sentences
                                          dailyMedDisContentStrList = tokenize.sent_tokenize(contentStr)
                                          #print(dailyMedDisContentStrList[0])

                                          for st in dailyMedDisContentStrList:
                                            #print(st)

                                            geneProtein = []
                                            combDrug = []
                                            negGeProDrList = []
                                            
                                            # do NER and extract gene and protein (without considering logical structure)
                                            geneProteinDic = biomarker_extraction.gene_protein_chemical(text = st, gene= 1, protein = 1)
                                            geneProtein.extend(geneProteinDic.get("gene"))
                                            geneProtein.extend(geneProteinDic.get("protein"))
                                            #print(geneProtein)

                                            # check first-line
                                            fLine = ''
                                            if biomarker_extraction.is_firstline(text = st, medicine = brName, disease= d):
                                              fLine = 'Yes'

                                            # check metastatic
                                            met = ''
                                            if biomarker_extraction.is_metastatic(text = st, disease = d):
                                              met = 'Yes'
                                            
                                            # do 'in conbination with' and 'used with'
                                            subTree = biomarker_extraction.sent_subtree(st)
                                            if len(subTree) >0:
                                              subTreeConca = ' '.join(tree for tree in subTree)
                                              drug = biomarker_extraction.gene_protein_chemical(text = subTreeConca, chemical = 1)
                                              combDrug.extend(drug.get("chemical"))
                                              # remove duplicates
                                              combDrug = list(set(combDrug))
                                            
                                            # do negation and extract negated gene, protein, and drug
                                            negScopeStr = negation_cue_scope.negation_scope(text = st, modelCue = modelCue, modelScope = modelScope)
                                            #print(negScopeStr)
                                          
                                            geneProDrDic = {}
                                            if len(negScopeStr)>0:
                                              scopeStrConca = ' '.join(ng for ng in negScopeStr)
                                              geneProDrDic = biomarker_extraction.gene_protein_chemical(text=scopeStrConca, gene= 1, protein = 1, chemical = 1)

                                            #print(geneProDrDic)
                                            if len(geneProDrDic)>0:
                                              #print(geneProDrDic)
                                              negGeProDrList.extend(geneProDrDic.get("chemical"))
                                              negGeProDrList.extend(geneProDrDic.get("gene"))
                                              negGeProDrList.extend(geneProDrDic.get("protein"))
                                              negGeProDrList = list(set(negGeProDrList))
                                              
                                              # clean (remove brand name fromthe negation list if there is)
                                              if brName in negGeProDrList:
                                                negGeProDrList.remove(brName)

                                            # add the biomarkers to lists

                                            if len(geneProtein)> 0:
                                              
                                              rowList = list(set(geneProtein))
                                              
                                              # clean (remove brand label and special characters from gene and protein list)
                                              noisyChar = [brName, '®', '™', brName+'®', brName+'™' ]
                                              removeList = []
                                              for ro in rowList:
                                                for nc in noisyChar:
                                                  if nc in ro:
                                                    removeList.append(ro)
                                              cleanList =[ro for ro in rowList if ro not in removeList]

                                              if len(cleanList) > 0:
                                                for gp in cleanList:
                                                  geneProtein_label.append(gp)
                                                  drug_label.append(therBrName)
                                                  therapy_label.append(therName)
                                                  NDCCode_label.append(NDCCodes)
                                                  disease_label.append(d)

                                                  combDrugStr = ', '.join(c for c in combDrug if c)
                                                  combinationDrug_label.append(combDrugStr)

                                                  negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                                  negGeProDr_label.append(negGeProDrListStr)
                                                  
                                                  firstLine.append(fLine)
                                                  acceleratedApproval.append(accAppr)
                                                  acceleratedApprovalRate.append(accApprRate)
                                                  metastatic.append(met)
                                              else:
                                                # exclude the unnecessary rows if other disease related data already exits
                                                colList = [len(combDrug)>0, len(negGeProDrList)>0, fLine == 'Yes', accAppr == 'Yes', accApprRate == 'Yes', met == 'Yes']
                                                if any(colList):
                                                  geneProtein_label.append("*")
                                                  drug_label.append(therBrName)
                                                  therapy_label.append(therName)
                                                  NDCCode_label.append(NDCCodes)
                                                  disease_label.append(d)

                                                  combDrugStr = ', '.join(c for c in combDrug if c)
                                                  combinationDrug_label.append(combDrugStr)

                                                  negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                                  negGeProDr_label.append(negGeProDrListStr)
                                                  
                                                  firstLine.append(fLine)
                                                  acceleratedApproval.append(accAppr)
                                                  acceleratedApprovalRate.append(accApprRate)
                                                  metastatic.append(met)
                                            
                                            else:
                                              geneProtein_label.append("*")
                                              drug_label.append(therBrName)
                                              therapy_label.append(therName)
                                              NDCCode_label.append(NDCCodes)
                                              disease_label.append(d)

                                              combDrugStr = ', '.join(c for c in combDrug if c)
                                              combinationDrug_label.append(combDrugStr)

                                              negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                              negGeProDr_label.append(negGeProDrListStr)
                                              
                                              firstLine.append(fLine)
                                              acceleratedApproval.append(accAppr)
                                              acceleratedApprovalRate.append(accApprRate)
                                              metastatic.append(met)

                                        # not text information related to the disease
                                        else: 
                                          geneProtein_label.append("*")
                                          drug_label.append(therBrName)
                                          therapy_label.append(therName)
                                          NDCCode_label.append(NDCCodes)
                                          disease_label.append(d)
                                          combinationDrug_label.append("")
                                          negGeProDr_label.append("")
                                          firstLine.append("")
                                          acceleratedApproval.append("")
                                          acceleratedApprovalRate.append("")
                                          metastatic.append("")

                                    # no diseases are detected
                                    else:
                                      geneProtein_label.append("*")
                                      drug_label.append(therBrName)
                                      therapy_label.append(therName)
                                      NDCCode_label.append(NDCCodes)
                                      disease_label.append("")
                                      combinationDrug_label.append("")
                                      negGeProDr_label.append("")
                                      firstLine.append("")
                                      acceleratedApproval.append("")
                                      acceleratedApprovalRate.append("")
                                      metastatic.append("")

                            
                        ## drug information page
                        elif 'dailymed/drugInfo' in dailyMedURL:
                          #print(dailyMedURL)
                          print('single choice')
                          
                          # extract the brand name
                          therBrName = biomarker_extraction.drug_brand_label(dailyMedURL = dailyMedURL)
                          brName = therBrName.split("-")[0]

                          # extract the NDC code
                          NDCCodes = biomarker_extraction.ndc_code(dailyMedURL = dailyMedURL)

                          # diseases are detected
                          if len(diseaseList) > 0:

                            for d in diseaseList:
                              print(d)

                              # locate 'INDICATIONS AND USAGE' section on DailyMed and extract the text content of a certain cancer type. 
                              # Use '\t' or ' ' + d to handle issue of disease with Non in front, e.g.'Non-Small Cell Lung Cancer' and 'Small Cell Lung Cancer'
                              # 'Non-' in the disease list
                              if any('non-' + d.lower() in dl.lower() for dl in diseaseList):
                                dailyMedDisContentStr_1 = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = '\t' + d, header= False)
                                dailyMedDisContentStr_2 = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = ' ' + d, header= False)
                                if dailyMedDisContentStr_1:
                                  dailyMedDisContentStr = dailyMedDisContentStr_1
                                elif dailyMedDisContentStr_2:
                                  dailyMedDisContentStr = dailyMedDisContentStr_2
                                else:
                                  dailyMedDisContentStr = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = d, header= False)

                              # 'Non-' not in the disease list 
                              else: dailyMedDisContentStr = biomarker_extraction.disease_content(dailyMedURL = dailyMedURL, disease = d, header= False)

                              # get the whole section text of 'INDICATIONS AND USAGE'
                              sectionContent = biomarker_extraction.section_content(dailyMedURL, section = 'INDICATIONS AND USAGE')

                              contentStr = None
                              if dailyMedDisContentStr:
                                #print(dailyMedDisContentStr)
                                contentStr = dailyMedDisContentStr

                              else:
                                if sectionContent:
                                  if d.lower() in sectionContent.lower():
                                    contentStr = sectionContent

                              # text information related to the disease
                              if contentStr:
                                print('has disease content')
                                # check if it is accelerated approval and by response rate
                                accApprRate = ''
                                if biomarker_extraction.is_accelerated_approval_rate(contentStr):
                                  accApprRate = 'Yes'

                                # check if it is accelerated approval
                                accAppr = ''
                                if biomarker_extraction.is_accelerated_approval(contentStr):
                                  accAppr = 'Yes'

                                # split into sentences
                                dailyMedDisContentStrList = tokenize.sent_tokenize(contentStr)
                                #print(dailyMedDisContentStrList[0])

                                for st in dailyMedDisContentStrList:
                                  #print(st)

                                  geneProtein = []
                                  combDrug = []
                                  negGeProDrList = []
                                  
                                  # do NER and extract gene and protein (without considering logical structure)
                                  geneProteinDic = biomarker_extraction.gene_protein_chemical(text = st, gene= 1, protein = 1)
                                  geneProtein.extend(geneProteinDic.get("gene"))
                                  geneProtein.extend(geneProteinDic.get("protein"))
                                  #print(geneProtein)

                                  # check first-line
                                  fLine = ''
                                  if biomarker_extraction.is_firstline(text = st, medicine = brName, disease= d):
                                    fLine = 'Yes'

                                  # check metastatic
                                  met = ''
                                  if biomarker_extraction.is_metastatic(text = st, disease = d):
                                    met = 'Yes'
                                  
                                  # do 'in conbination with' and 'used with'
                                  subTree = biomarker_extraction.sent_subtree(st)
                                  if len(subTree) >0:
                                    subTreeConca = ' '.join(tree for tree in subTree)
                                    drug = biomarker_extraction.gene_protein_chemical(text = subTreeConca, chemical = 1)
                                    combDrug.extend(drug.get("chemical"))
                                    # remove duplicates
                                    combDrug = list(set(combDrug))
                                  
                                  # do negation and extract negated gene, protein, and drug
                                  negScopeStr = negation_cue_scope.negation_scope(text = st, modelCue = modelCue, modelScope = modelScope)
                                  #print(negScopeStr)
                                
                                  geneProDrDic = {}
                                  if len(negScopeStr)>0:
                                    scopeStrConca = ' '.join(ng for ng in negScopeStr)
                                    geneProDrDic = biomarker_extraction.gene_protein_chemical(text=scopeStrConca, gene= 1, protein = 1, chemical = 1)

                                  #print(geneProDrDic)
                                  if len(geneProDrDic)>0:
                                    #print(geneProDrDic)
                                    negGeProDrList.extend(geneProDrDic.get("chemical"))
                                    negGeProDrList.extend(geneProDrDic.get("gene"))
                                    negGeProDrList.extend(geneProDrDic.get("protein"))
                                    negGeProDrList = list(set(negGeProDrList))
                                    
                                    # clean (remove brand name fromthe negation list if there is)
                                    if brName in negGeProDrList:
                                      negGeProDrList.remove(brName)

                                  # add the biomarkers to lists

                                  if len(geneProtein)> 0:
                                    
                                    rowList = list(set(geneProtein))
                                    #print(rowList)
                                    
                                    # clean (remove brand label and special characters from gene and protein list)
                                    noisyChar = [brName, '®', '™', brName+'®', brName+'™' ]
                                    removeList = []
                                    for ro in rowList:
                                      for nc in noisyChar:
                                        if nc in ro:
                                          removeList.append(ro)
                                    cleanList =[ro for ro in rowList if ro not in removeList]
                                    
                                    #print(cleanList)

                                    if len(cleanList) > 0:
                                      for gp in cleanList:
                                        geneProtein_label.append(gp)
                                        drug_label.append(therBrName)
                                        therapy_label.append(therName)
                                        NDCCode_label.append(NDCCodes)
                                        disease_label.append(d)

                                        combDrugStr = ', '.join(c for c in combDrug if c)
                                        combinationDrug_label.append(combDrugStr)

                                        negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                        negGeProDr_label.append(negGeProDrListStr)
                                        
                                        firstLine.append(fLine)
                                        acceleratedApproval.append(accAppr)
                                        acceleratedApprovalRate.append(accApprRate)
                                        metastatic.append(met)
                                    else:
                                      # exclude the unnecessary rows if other disease related data already exits
                                      colList = [len(combDrug)>0, len(negGeProDrList)>0, fLine == 'Yes', accAppr == 'Yes', accApprRate == 'Yes', met == 'Yes']
                                      if any(colList):
                                        geneProtein_label.append("*")
                                        drug_label.append(therBrName)
                                        therapy_label.append(therName)
                                        NDCCode_label.append(NDCCodes)
                                        disease_label.append(d)

                                        combDrugStr = ', '.join(c for c in combDrug if c)
                                        combinationDrug_label.append(combDrugStr)

                                        negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                        negGeProDr_label.append(negGeProDrListStr)
                                        
                                        firstLine.append(fLine)
                                        acceleratedApproval.append(accAppr)
                                        acceleratedApprovalRate.append(accApprRate)
                                        metastatic.append(met)
                                  
                                  else:
                                    geneProtein_label.append("*")
                                    drug_label.append(therBrName)
                                    therapy_label.append(therName)
                                    NDCCode_label.append(NDCCodes)
                                    disease_label.append(d)

                                    combDrugStr = ', '.join(c for c in combDrug if c)
                                    combinationDrug_label.append(combDrugStr)

                                    negGeProDrListStr = ', '.join(n for n in negGeProDrList if n)
                                    negGeProDr_label.append(negGeProDrListStr)
                                    
                                    firstLine.append(fLine)
                                    acceleratedApproval.append(accAppr)
                                    acceleratedApprovalRate.append(accApprRate)
                                    metastatic.append(met)

                              # not text information related to the disease
                              else: 
                                geneProtein_label.append("*")
                                drug_label.append(therBrName)
                                therapy_label.append(therName)
                                NDCCode_label.append(NDCCodes)
                                disease_label.append(d)
                                combinationDrug_label.append("")
                                negGeProDr_label.append("")
                                firstLine.append("")
                                acceleratedApproval.append("")
                                acceleratedApprovalRate.append("")
                                metastatic.append("")

                          # no diseases are detected
                          else:
                            geneProtein_label.append("*")
                            drug_label.append(therBrName)
                            therapy_label.append(therName)
                            NDCCode_label.append(NDCCodes)
                            disease_label.append("")
                            combinationDrug_label.append("")
                            negGeProDr_label.append("")
                            firstLine.append("")
                            acceleratedApproval.append("")
                            acceleratedApprovalRate.append("")
                            metastatic.append("")
      
    

Atezolizumab
1
['Urothelial carcinoma', 'Small cell lung cancer', 'Melanoma', 'Non-small cell lung cancer', 'Hepatocellular carcinoma', 'Breast cancer']
single choice
Urothelial carcinoma
has disease content


/usr/local/lib/python3.7/dist-packages/biomarker_nlp/negation_negbert.py:2632: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode_pred = np.argmax(np.average(np.array(curr_preds), axis=0), axis=0)


/usr/local/lib/python3.7/dist-packages/biomarker_nlp/negation_negbert.py:3016: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode_pred = np.argmax(np.average(np.array(curr_preds), axis=0), axis=0)
/usr/local/lib/python3.7/dist-packages/biomarker_nlp/negation_negbert.py:2646: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode_pred = np.argmax(np.average(np.array(curr_preds), axis=0), axis=0)


Small cell lung cancer
has disease content
Melanoma
has disease content
Non-small cell lung cancer
has disease content


/usr/local/lib/python3.7/dist-packages/biomarker_nlp/negation_negbert.py:3030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode_pred = np.argmax(np.average(np.array(curr_preds), axis=0), axis=0)


Hepatocellular carcinoma
has disease content
Breast cancer
has disease content
Nivolumab
2
['Malignant pleural mesothelioma', 'Squamous cell carcinoma of the head and neck', 'Urothelial carcinoma', 'Renal cell carcinoma', 'gastroesophageal junction adenocarcinoma', 'Classic Hodgkin lymphoma', 'Melanoma', 'Non-small cell lung cancer', 'Hepatocellular carcinoma', 'esophageal cancer', 'Gastric (stomach) cancer', 'Colorectal cancer', 'Esophageal cancer or gastroesophageal junction adenocarcinoma']
single choice
Malignant pleural mesothelioma
has disease content
Squamous cell carcinoma of the head and neck
has disease content
Urothelial carcinoma
has disease content
Renal cell carcinoma
has disease content
gastroesophageal junction adenocarcinoma
Classic Hodgkin lymphoma
Melanoma
has disease content
Non-small cell lung cancer
has disease content
Hepatocellular carcinoma
has disease content
esophageal cancer
has disease content
Gastric (stomach) cancer
Colorectal cancer
has disease content
E

In [ ]:
len(therapy_list), len(geneProtein_label), len(drug_label), len(therapy_label), len(NDCCode_label), len(disease_label), len(combinationDrug_label), len(negGeProDr_label), len(firstLine), len(acceleratedApproval), len(acceleratedApprovalRate), len(metastatic)

(152, 2144, 2144, 2144, 2144, 2144, 2144, 2144, 2144, 2144, 2144, 2144)

In [ ]:
# put the label's lists into a data frame
dict = {'geneProtein_label': geneProtein_label, 'therapy_label':therapy_label, 'disease_label': disease_label, 'drug_label': drug_label, 'NDCCode_label': NDCCode_label, 'combinationDrug_label':combinationDrug_label, 'negGeProDr_label':negGeProDr_label, 'firstLine':firstLine, 'acceleratedApproval': acceleratedApproval, 'acceleratedApprovalRate':acceleratedApprovalRate, 'metastatic':metastatic}  
therapyBiomarker = pd.DataFrame(dict)
therapyBiomarker.head()

,geneProtein_label,therapy_label,disease_label,drug_label,NDCCode_label,combinationDrug_label,negGeProDr_label,firstLine,acceleratedApproval,acceleratedApprovalRate,metastatic
0,PD-L1,Atezolizumab,Urothelial carcinoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,,,Yes,Yes,Yes
1,*,Atezolizumab,Urothelial carcinoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,,,Yes,Yes,
2,*,Atezolizumab,Small cell lung cancer,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...","carboplatin, etoposide",,Yes,,,
3,BRAF,Atezolizumab,Melanoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",vemurafenib,,,,,Yes
4,PD-L1,Atezolizumab,Non-small cell lung cancer,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,"EGFR, ALK genomic",Yes,,,Yes


In [ ]:
# drop duplicate rows
therapyBiomarker_final = therapyBiomarker.drop_duplicates(keep = 'first').reset_index().drop('index', axis=1)
therapyBiomarker_final.head()

,geneProtein_label,therapy_label,disease_label,drug_label,NDCCode_label,combinationDrug_label,negGeProDr_label,firstLine,acceleratedApproval,acceleratedApprovalRate,metastatic
0,PD-L1,Atezolizumab,Urothelial carcinoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,,,Yes,Yes,Yes
1,*,Atezolizumab,Urothelial carcinoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,,,Yes,Yes,
2,*,Atezolizumab,Small cell lung cancer,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...","carboplatin, etoposide",,Yes,,,
3,BRAF,Atezolizumab,Melanoma,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",vemurafenib,,,,,Yes
4,PD-L1,Atezolizumab,Non-small cell lung cancer,"TECENTRIQ- atezolizumab injection, solution","50242-917-01, 50242-917-86, 50242-918-01, 5024...",,"EGFR, ALK genomic",Yes,,,Yes


In [ ]:
therapyBiomarker_final.shape

(1210, 11)

In [ ]:
# save the data frame to Google drive.  
# therapyBiomarker_final.to_csv(r'/path/to/the/location/on/google/drive/filename.csv')